In [5]:
!git clone https://github.com/pthengtr/kcw-analytics.git

fatal: destination path 'kcw-analytics' already exists and is not an empty directory.


In [6]:
!cd /content/kcw-analytics && git pull origin main

From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
Already up to date.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import pandas as pd

folder = "/content/drive/MyDrive/kcw_analytics/01_raw"

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")



Loaded: raw_inventory_hq_2024.csv -> (4983, 8)
Loaded: raw_hq_icmas_products.csv -> (114863, 94)
Loaded: raw_hq_pidet_purchase_lines.csv -> (248629, 41)
Loaded: raw_hq_pimas_purchase_bills.csv -> (83359, 49)
Loaded: raw_hq_sidet_sales_lines.csv -> (1198025, 38)
Loaded: raw_hq_simas_sales_bills.csv -> (485483, 49)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2866, 49)
Loaded: raw_syp_simas_sales_bills.csv -> (11750, 49)
Loaded: raw_syp_pidet_purchase_lines.csv -> (26781, 41)
Loaded: raw_syp_sidet_sales_lines.csv -> (34635, 38)


In [9]:
hq_sales_lines = data['raw_hq_sidet_sales_lines.csv'].copy()
syp_sales_lines = data['raw_syp_sidet_sales_lines.csv'].copy()
purchase_lines = data['raw_hq_pidet_purchase_lines.csv'].copy()

In [10]:
hq_sales_lines.columns

Index(['ID', 'JOURMODE', 'JOURTYPE', 'JOURDATE', 'BILLTYPE', 'BILLDATE',
       'BILLNO', 'LINE', 'ITEMNO', 'BCODE', 'PCODE', 'MCODE', 'DETAIL',
       'WHNUMBER', 'LOCATION1', 'STATUS', 'SERIAL', 'TAXIC', 'EXMPT', 'ISVAT',
       'QTY', 'UI', 'MTP', 'PRICE', 'XPRICE', 'DISCNT1', 'DISCNT2', 'DISCNT3',
       'DISCNT4', 'DED', 'VAT', 'AMOUNT', 'CHGAMT', 'ACCTNO', 'PAID',
       'ACCT_NO', 'DONE', 'CANCELED'],
      dtype='object')

In [11]:
import pandas as pd
import numpy as np

def refill_last_cost_from_icmas(
    data: dict,
    df: pd.DataFrame,
    *,
    icmas_key: str = "raw_hq_icmas_products.csv",
    bcode_col: str = "BCODE",
    last_cost_col: str = "LAST_COST",
    icmas_cost_col: str = "COSTNET",
) -> pd.DataFrame:
    """
    Refill LAST_COST when it is 0 or NaN using COSTNET from ICMAS.
    Includes BCODE cleanup to avoid merge mismatch.
    """

    result = df.copy()

    # --- BCODE CLEANUP (VERY IMPORTANT for KCW datasets) ---
    result[bcode_col] = (
        result[bcode_col]
        .astype(str)
        .str.strip()
        .str.upper()
    )

    icmas = data[icmas_key][[bcode_col, icmas_cost_col]].copy()

    icmas[bcode_col] = (
        icmas[bcode_col]
        .astype(str)
        .str.strip()
        .str.upper()
    )

    # --- numeric safety ---
    result[last_cost_col] = pd.to_numeric(result[last_cost_col], errors="coerce")
    icmas[icmas_cost_col] = pd.to_numeric(icmas[icmas_cost_col], errors="coerce")

    # --- merge COSTNET ---
    result = result.merge(
        icmas,
        on=bcode_col,
        how="left",
        suffixes=("", "_ICMAS")
    )

    # --- detect invalid LAST_COST ---
    mask_invalid = result[last_cost_col].isna() | (result[last_cost_col] == 0)

    # --- refill only invalid rows ---
    result.loc[mask_invalid, last_cost_col] = result.loc[
        mask_invalid, icmas_cost_col
    ]

    # --- drop helper column ---
    result = result.drop(columns=[icmas_cost_col])

    return result

def qc_unknown(df, label):
    total = len(df)
    unk = (df["COST_STATUS"] == "UNKNOWN").sum()
    print(f"[{label}] UNKNOWN: {unk:,} / {total:,} ({(unk/total*100 if total else 0):.2f}%)")

In [12]:
import numpy as np
import pandas as pd

_BCODE_RE = r"^\d{8}$"

def _to_numeric_clean(series: pd.Series) -> pd.Series:
    """
    Convert common messy numeric strings to numbers.
    Handles: whitespace, NBSP, commas.
    Non-convertible -> NaN.
    """
    s = series.astype("string").str.strip()
    s = s.str.replace("\u00A0", " ", regex=False)  # non-breaking space
    s = s.str.replace(",", "", regex=False)        # "1,234.50" -> "1234.50"
    return pd.to_numeric(s, errors="coerce")


def add_sales_quality_flags(
    df: pd.DataFrame,
    *,
    bcode_col: str = "BCODE",
    price_col: str = "PRICE",
    amount_col: str = "AMOUNT",
    canceled_col: str = "CANCELED",
    add_row_id: bool = True,
) -> pd.DataFrame:
    """
    Adds Power-BI-friendly numeric columns + data-quality flags, without removing rows.

    Output columns added:
      - BCODE (trimmed)
      - PRICE_NUM, AMOUNT_NUM (numeric)
      - IS_VALID (bool)
      - INVALID_REASON (text, e.g. "BAD_BCODE|BAD_AMOUNT")
      - ROW_ID (optional)
    """
    out = df.copy()

    # --- BCODE clean + valid ---
    b = out[bcode_col].astype("string").str.strip()
    out[bcode_col] = b
    b_ok = b.fillna("").str.match(_BCODE_RE)

    # --- numeric clean (keep raw, add numeric columns) ---
    price_num = _to_numeric_clean(out[price_col])
    amount_num = _to_numeric_clean(out[amount_col])

    # Power BI hates inf/-inf
    price_ok = price_num.notna() & np.isfinite(price_num.to_numpy())
    amount_ok = amount_num.notna() & np.isfinite(amount_num.to_numpy())

    out[f"{price_col}_NUM"] = price_num
    out[f"{amount_col}_NUM"] = amount_num

    # --- canceled flag ---
    c = out[canceled_col].astype("string").str.strip().str.upper()
    canceled_ok = c != "Y"

    # --- overall validity ---
    out["IS_VALID"] = b_ok & price_ok & amount_ok & canceled_ok

    # --- reason text (can contain multiple reasons) ---
    reason = pd.Series("", index=out.index, dtype="string")

    def add_reason(mask, label):
        nonlocal reason
        reason = np.where(
            mask,
            np.where(reason == "", label, reason + "|" + label),
            reason
        )
        reason = pd.Series(reason, index=out.index, dtype="string")

    add_reason(~b_ok, "BAD_BCODE")
    add_reason(~price_ok, "BAD_PRICE")
    add_reason(~amount_ok, "BAD_AMOUNT")
    add_reason(~canceled_ok, "CANCELED")

    out["INVALID_REASON"] = reason.replace("", pd.NA)

    if add_row_id and "ROW_ID" not in out.columns:
        out["ROW_ID"] = np.arange(len(out), dtype=np.int64)

    return out


In [13]:
!pip install tqdm

In [14]:
def enrich_sales_with_last_purchase_cost(
    sales: pd.DataFrame,
    purchases: pd.DataFrame,
    *,
    bcode_col: str = "BCODE",
    sale_date_col: str = "BILLDATE",
    purch_date_col: str = "BILLDATE",
    qty_col: str = "QTY",
    mtp_col: str = "MTP",
    amount_col: str = "AMOUNT",
    out_cost_col: str = "LAST_PURCHASE_COST",
    out_pdate_col: str = "LAST_PURCHASE_DATE",
    out_status_col: str = "COST_STATUS",
) -> pd.DataFrame:

    s = sales.copy()
    p = purchases.copy()

    s[bcode_col] = s[bcode_col].astype("string").str.strip()
    p[bcode_col] = p[bcode_col].astype("string").str.strip()

    s[sale_date_col] = pd.to_datetime(s[sale_date_col], errors="coerce")
    p[purch_date_col] = pd.to_datetime(p[purch_date_col], errors="coerce")

    denom = p[qty_col].astype(float) * p[mtp_col].astype(float)
    p["_UNIT_COST"] = np.where(denom != 0, p[amount_col].astype(float) / denom, np.nan)

    # Keep only valid purchases
    p = p[p[purch_date_col].notna() & p["_UNIT_COST"].notna()].copy()

    # ✅ Create a separate right-side date column so we don't overwrite sale BILLDATE
    p["_PURCH_DATE"] = p[purch_date_col]

    s["_POS"] = np.arange(len(s))
    s_valid = s[s[sale_date_col].notna()].copy()
    s_invalid = s[s[sale_date_col].isna()].copy()

    s_valid = s_valid.sort_values([sale_date_col, bcode_col, "_POS"], kind="mergesort")
    p = p.sort_values(["_PURCH_DATE", bcode_col], kind="mergesort")

    merged = pd.merge_asof(
        s_valid,
        p[[bcode_col, "_PURCH_DATE", "_UNIT_COST"]],
        left_on=sale_date_col,
        right_on="_PURCH_DATE",
        by=bcode_col,
        direction="backward",
        allow_exact_matches=True,
    )

    merged.rename(columns={"_UNIT_COST": out_cost_col, "_PURCH_DATE": out_pdate_col}, inplace=True)
    merged[out_status_col] = np.where(merged[out_cost_col].notna(), "OK", "UNKNOWN")

    if len(s_invalid) > 0:
        s_invalid[out_cost_col] = np.nan
        s_invalid[out_pdate_col] = pd.NaT
        s_invalid[out_status_col] = "UNKNOWN"
        merged = pd.concat([merged, s_invalid], ignore_index=False)

    merged = merged.sort_values("_POS", kind="mergesort").drop(columns=["_POS"])
    return merged


In [15]:
syp_sales_flagged = add_sales_quality_flags(syp_sales_lines)

# QC summary (no deletion)
total = len(syp_sales_flagged)
invalid = (~syp_sales_flagged["IS_VALID"]).sum()
print(f"Invalid: {invalid:,}/{total:,} ({invalid/total*100:.2f}%)")

print(
    syp_sales_flagged["INVALID_REASON"]
    .fillna("OK")
    .value_counts()
    .head(20)
)

# For analytics (optional): just filter in pandas
syp_sales_valid_only = syp_sales_flagged[syp_sales_flagged["IS_VALID"]].copy()


Invalid: 13/34,635 (0.04%)
INVALID_REASON
OK            34622
BAD_BCODE         7
BAD_AMOUNT        6
Name: count, dtype: Int64


In [16]:
hq_sales_flagged = add_sales_quality_flags(hq_sales_lines)

# QC summary (no deletion)
total = len(hq_sales_flagged)
invalid = (~hq_sales_flagged["IS_VALID"]).sum()
print(f"Invalid: {invalid:,}/{total:,} ({invalid/total*100:.2f}%)")

print(
    hq_sales_flagged["INVALID_REASON"]
    .fillna("OK")
    .value_counts()
    .head(20)
)

# For analytics (optional): just filter in pandas
syp_sales_valid_only = hq_sales_flagged[hq_sales_flagged["IS_VALID"]].copy()

Invalid: 31,334/1,198,025 (2.62%)
INVALID_REASON
OK                                1166691
BAD_BCODE                           28571
CANCELED                             1806
BAD_AMOUNT                            404
BAD_BCODE|BAD_PRICE                   280
BAD_PRICE                             132
BAD_BCODE|BAD_PRICE|BAD_AMOUNT         94
BAD_PRICE|BAD_AMOUNT                   29
BAD_BCODE|CANCELED                     17
BAD_AMOUNT|CANCELED                     1
Name: count, dtype: Int64


In [17]:
hq_sales_enriched = enrich_sales_with_last_purchase_cost(
    hq_sales_flagged,
    purchase_lines,
)

qc_unknown(hq_sales_enriched, "before refill")

hq_sales_enriched = refill_last_cost_from_icmas(
    data,
    hq_sales_enriched,
    last_cost_col="LAST_PURCHASE_COST",
)

qc_unknown(hq_sales_enriched, "after refill")



[before refill] UNKNOWN: 69,136 / 1,198,025 (5.77%)
[after refill] UNKNOWN: 69,136 / 1,198,025 (5.77%)


In [18]:
syp_sales_enriched = enrich_sales_with_last_purchase_cost(
    syp_sales_flagged,
    purchase_lines,
)

qc_unknown(syp_sales_enriched, "before refill")

syp_sales_enriched = refill_last_cost_from_icmas(
    data,
    syp_sales_enriched,
    last_cost_col="LAST_PURCHASE_COST",
)

qc_unknown(syp_sales_enriched, "after refill")

[before refill] UNKNOWN: 508 / 34,635 (1.47%)
[after refill] UNKNOWN: 508 / 34,635 (1.47%)


In [19]:
hq_sales_enriched["BRANCH"] = "HQ"
syp_sales_enriched["BRANCH"] = "SYP"

In [20]:
sales_all = pd.concat([hq_sales_enriched, syp_sales_enriched], ignore_index=True)

sales_all["BRANCH"] = sales_all["BRANCH"].astype("string")
sales_all["LAST_PURCHASE_COST"] = pd.to_numeric(sales_all["LAST_PURCHASE_COST"], errors="coerce")
sales_all["BILLDATE"] = pd.to_datetime(sales_all["BILLDATE"], errors="coerce")

sales_all["BRANCH_BILLNO"] = sales_all["BRANCH"] + "-" + sales_all["BILLNO"].astype(str)


In [21]:
sales_all["BILLTYPE_STD"] = (
    sales_all["BILLNO"]
    .astype("string")
    .str.upper()
    .str.replace(r"^3", "", regex=True)   # remove leading 3 if exists
    .str.extract(r"^(TFV|TAD|TAR|TR|TD|TF|CN|DN)", expand=False)
    .fillna("UNKNOWN")
)

In [22]:
KEEP_COLS = [
    'BILLDATE', 'BILLTYPE', 'JOURMODE',
    'BILLNO', 'BCODE', 'DETAIL',
    'STATUS', 'ISVAT',
    'QTY', 'UI', 'MTP', 'PRICE', 'XPRICE', 'DISCNT1', 'DISCNT2', 'DISCNT3',
    'DISCNT4', 'DED', 'VAT', 'AMOUNT', 'ACCTNO', 'PAID',
    'ACCT_NO', 'DONE', 'CANCELED',
    'PRICE_NUM', 'AMOUNT_NUM', 'IS_VALID', 'INVALID_REASON', 'ROW_ID',
    'LAST_PURCHASE_DATE', 'LAST_PURCHASE_COST', 'COST_STATUS',
    'BRANCH', 'BRANCH_BILLNO', 'BILLTYPE_STD'
]

# keep only columns that actually exist (prevents KeyError)
sales_all = sales_all[[c for c in KEEP_COLS if c in sales_all.columns]].copy()

In [23]:
sales_all

,BILLDATE,BILLTYPE,JOURMODE,BILLNO,BCODE,DETAIL,STATUS,ISVAT,QTY,UI,...,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH,BRANCH_BILLNO,BILLTYPE_STD
0,2015-06-20,1.0,2,KC1506-0006,<NA>,สลักปีกนก,1.0,N,1.0,ตัว,...,270.0,False,BAD_BCODE,0,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
1,2015-06-20,1.0,2,KC1506-0006,<NA>,ลูกหมากปีกนกแท้ D-MAX,1.0,N,1.0,ตัว,...,750.0,False,BAD_BCODE,1,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
2,2015-06-20,1.0,2,KC1506-0006,<NA>,แผ่นผ้าทราย,1.0,N,2.0,แผ่น,...,30.0,False,BAD_BCODE,2,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
3,2015-06-20,1.0,2,KC1506-0006,<NA>,ยางกันโครง,1.0,N,1.0,ตัว,...,20.0,False,BAD_BCODE,3,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
4,2015-06-20,1.0,2,KC1506-0006,<NA>,ยางรองคอยสปริง,1.0,N,2.0,ตัว,...,140.0,False,BAD_BCODE,4,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232655,2026-02-14,1.0,0,3TAR6902-346,03052667,"ลูกหมากปีกนกล่าง 2WD-4WD D-MAX,MU7.04-13,MU-X",1.0,Y,1.0,หน่วย,...,560.0,True,<NA>,34630,2026-01-26,372.100,OK,SYP,SYP-3TAR6902-346,TAR
1232656,2026-02-14,1.0,0,3TAR6902-346,11051952,เซ็นเซอร์ ABS หน้า ปาเจโร่ 2012 LH,1.0,Y,1.0,อัน,...,2450.0,True,<NA>,34631,2026-02-13,1501.500,OK,SYP,SYP-3TAR6902-346,TAR
1232657,2026-02-16,1.0,2,33K69-0001186,13013211,หลอดไฟในกันชน เขี้ยวตรง 12 V. 2 จุด สีส้ม,1.0,N,2.0,หลอด*,...,60.0,True,<NA>,34632,2026-02-06,16.000,OK,SYP,SYP-33K69-0001186,UNKNOWN
1232658,2026-02-16,1.0,2,33K69-0001187,08056145,"คอยล์เย็น VIOS,YARIS 2013",1.0,N,1.0,ลูก,...,1150.0,True,<NA>,34633,2025-11-01,747.665,OK,SYP,SYP-33K69-0001187,UNKNOWN


In [24]:
import pandas as pd
import numpy as np

def _clean_str(s: pd.Series) -> pd.Series:
    s = s.astype("string")

    # remove non-breaking space
    s = s.str.replace("\u00A0", " ", regex=False)

    # trim + normalize case
    s = s.str.strip().str.upper()

    # remove common fake-null strings
    s = s.replace({
        "": pd.NA,
        "NAN": pd.NA,
        "NONE": pd.NA,
        "NULL": pd.NA
    })

    return s



def _to_dt(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce")

# ------------------------
# DimDate
# ------------------------
def build_dim_date_from_sales(sales_all: pd.DataFrame, *, date_col: str = "BILLDATE") -> pd.DataFrame:
    d = _to_dt(sales_all[date_col]).dropna().dt.normalize()
    if d.empty:
        return pd.DataFrame(columns=["Date", "DateKey", "Year", "Month", "Day", "YearMonth", "Quarter", "WeekNum"])

    date_range = pd.date_range(d.min(), d.max(), freq="D")
    dim = pd.DataFrame({"Date": date_range})
    dim["DateKey"] = dim["Date"].dt.strftime("%Y%m%d").astype(int)
    dim["Year"] = dim["Date"].dt.year
    dim["Month"] = dim["Date"].dt.month
    dim["Day"] = dim["Date"].dt.day
    dim["YearMonth"] = dim["Date"].dt.strftime("%Y-%m")
    dim["Quarter"] = dim["Date"].dt.quarter
    dim["WeekNum"] = dim["Date"].dt.isocalendar().week.astype(int)
    return dim

# ------------------------
# DimBranch
# ------------------------
def build_dim_branch(sales_all: pd.DataFrame, *, branch_col: str = "BRANCH") -> pd.DataFrame:
    dim = pd.DataFrame({"BRANCH": _clean_str(sales_all[branch_col])}).dropna()

    dim = (
        dim[dim["BRANCH"] != ""]
        .drop_duplicates()
        .sort_values("BRANCH")
        .reset_index(drop=True)
    )

    # Keep your existing key
    dim["BranchKey"] = dim["BRANCH"]

    # 🔥 Map branch → UUID
    BRANCH_UUID_MAP = {
        "HQ": "c93efb5f-07c9-4229-b6b3-568ce1c0a9ab",
        "SYP": "4975a5a1-90e6-443a-9921-c6c637f4631c",
    }

    dim["branch_uuid"] = dim["BRANCH"].map(BRANCH_UUID_MAP)

    return dim


# ------------------------
# DimProduct (BCODE)
# ------------------------
def build_dim_product(
    sales_all: pd.DataFrame,
    *,
    bcode_col: str = "BCODE",
    detail_col: str = "DETAIL",
    ui_col: str = "UI",
    last_seen_date_col: str = "BILLDATE",
) -> pd.DataFrame:
    df = sales_all.copy()
    df[bcode_col] = _clean_str(df[bcode_col])
    df[detail_col] = _clean_str(df.get(detail_col, ""))
    df[ui_col] = _clean_str(df.get(ui_col, ""))
    df[last_seen_date_col] = _to_dt(df[last_seen_date_col])

    df = df[df[bcode_col].notna() & (df[bcode_col] != "")]
    df = df.sort_values([bcode_col, last_seen_date_col], kind="mergesort")
    last = df.groupby(bcode_col, sort=False).tail(1)

    dim = pd.DataFrame({
        "BCODE": last[bcode_col],
        "DETAIL": last.get(detail_col, pd.Series([pd.NA]*len(last))),
        "UI": last.get(ui_col, pd.Series([pd.NA]*len(last))),
        "LastSeenDate": last[last_seen_date_col].dt.normalize(),
    }).reset_index(drop=True)

    dim["ProductKey"] = dim["BCODE"]
    # add CATEGORY_CODE (first 2 digits) for easy relationship too
    dim["CATEGORY_CODE"] = dim["BCODE"].astype("string").str.slice(0, 2)
    return dim

# ------------------------
# DimCategory (first 2 digits of BCODE)
# ------------------------
def build_dim_category_from_bcode(
    sales_all: pd.DataFrame,
    *,
    bcode_col: str = "BCODE",
) -> pd.DataFrame:
    b = _clean_str(sales_all[bcode_col])
    cat = b.dropna().str.slice(0, 2)
    # keep only exactly 2 digits
    cat = cat[cat.str.match(r"^\d{2}$", na=False)]

    dim = pd.DataFrame({"CATEGORY_CODE": cat}).drop_duplicates().sort_values("CATEGORY_CODE").reset_index(drop=True)
    dim["CategoryKey"] = dim["CATEGORY_CODE"]
    return dim

# ------------------------
# DimAccount
# ------------------------
def build_dim_account(
    sales_all: pd.DataFrame,
    *,
    customer_col: str = "ACCTNO",
    supplier_col: str = "ACCT_NO",
) -> pd.DataFrame:
    # clean series (or empty)
    c = _clean_str(sales_all[customer_col]) if customer_col in sales_all.columns else pd.Series([], dtype="string")
    s = _clean_str(sales_all[supplier_col]) if supplier_col in sales_all.columns else pd.Series([], dtype="string")

    # unique set of accounts from both columns
    all_keys = pd.concat([c, s], ignore_index=True).dropna()
    all_keys = all_keys[all_keys != ""].drop_duplicates().sort_values().reset_index(drop=True)

    dim = pd.DataFrame({"AccountKey": all_keys})

    # role flags
    customer_set = set(c.dropna()[c.dropna() != ""].unique())
    supplier_set = set(s.dropna()[s.dropna() != ""].unique())

    dim["IsCustomer"] = dim["AccountKey"].isin(customer_set)
    dim["IsSupplier"] = dim["AccountKey"].isin(supplier_set)

    # optional: a readable label
    dim["AccountRole"] = np.where(dim["IsCustomer"] & dim["IsSupplier"], "BOTH",
                          np.where(dim["IsCustomer"], "CUSTOMER",
                          np.where(dim["IsSupplier"], "SUPPLIER", "UNKNOWN")))

    return dim


# ------------------------
# DimSupplier (ACCT_NO)
# ------------------------
def build_dim_supplier(sales_all: pd.DataFrame, *, supplier_col="ACCT_NO") -> pd.DataFrame:
    s = _clean_str(sales_all[supplier_col]) if supplier_col in sales_all.columns else pd.Series([pd.NA]*len(sales_all), dtype="string")
    dim = pd.DataFrame({"SupplierKey": s}).dropna()
    dim = dim[dim["SupplierKey"] != ""].drop_duplicates(subset=["SupplierKey"]).sort_values("SupplierKey").reset_index(drop=True)
    dim["SUPPLIER_ACCT_NO"] = dim["SupplierKey"]
    return dim

# -----------------------------
# DIM BILLTYPE (from BILLNO)
# -----------------------------
KNOWN_TYPES = ["TFV", "TAD", "TAR", "TR", "TD", "TF", "CN", "DN"]

def build_dim_billtype(sales_all):
    dim = pd.DataFrame({"BILLTYPE_STD": _clean_str(sales_all["BILLTYPE_STD"]).str.upper()})
    dim = dim.drop_duplicates().sort_values("BILLTYPE_STD").reset_index(drop=True)
    dim["BillTypeKey"] = dim["BILLTYPE_STD"]
    return dim

# ------------------------
# Wrapper
# ------------------------
def build_all_dims(sales_all):
    return {
        "dim_date": build_dim_date_from_sales(sales_all),
        "dim_product": build_dim_product(sales_all),
        "dim_category": build_dim_category_from_bcode(sales_all),
        "dim_account": build_dim_account(sales_all),
        "dim_branch": build_dim_branch(sales_all),
        "dim_billtype": build_dim_billtype(sales_all),
    }


In [25]:
dims = build_all_dims(sales_all)
{k: v.shape for k, v in dims.items()}

{'dim_date': (3895, 8),
 'dim_product': (34090, 6),
 'dim_category': (37, 2),
 'dim_account': (2948, 4),
 'dim_branch': (2, 3),
 'dim_billtype': (9, 2)}

In [26]:
import shutil
from pathlib import Path

out_dir = Path("/content/drive/MyDrive/kcw_analytics/03_curated")

if out_dir.exists():
    shutil.rmtree(out_dir)


In [27]:
import os
os.makedirs(out_dir, exist_ok=True)

for name, df in dims.items():
    df.to_csv(f"{out_dir}/{name}.csv", index=False, encoding="utf-8-sig")


In [28]:
sales_all.to_csv(
    f"{out_dir}/fact_sales_all.csv",
    index=False,
    encoding="utf-8-sig"   # important for Thai + Excel
)

**DEBUG**

In [29]:
sales_all

,BILLDATE,BILLTYPE,JOURMODE,BILLNO,BCODE,DETAIL,STATUS,ISVAT,QTY,UI,...,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH,BRANCH_BILLNO,BILLTYPE_STD
0,2015-06-20,1.0,2,KC1506-0006,<NA>,สลักปีกนก,1.0,N,1.0,ตัว,...,270.0,False,BAD_BCODE,0,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
1,2015-06-20,1.0,2,KC1506-0006,<NA>,ลูกหมากปีกนกแท้ D-MAX,1.0,N,1.0,ตัว,...,750.0,False,BAD_BCODE,1,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
2,2015-06-20,1.0,2,KC1506-0006,<NA>,แผ่นผ้าทราย,1.0,N,2.0,แผ่น,...,30.0,False,BAD_BCODE,2,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
3,2015-06-20,1.0,2,KC1506-0006,<NA>,ยางกันโครง,1.0,N,1.0,ตัว,...,20.0,False,BAD_BCODE,3,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
4,2015-06-20,1.0,2,KC1506-0006,<NA>,ยางรองคอยสปริง,1.0,N,2.0,ตัว,...,140.0,False,BAD_BCODE,4,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232655,2026-02-14,1.0,0,3TAR6902-346,03052667,"ลูกหมากปีกนกล่าง 2WD-4WD D-MAX,MU7.04-13,MU-X",1.0,Y,1.0,หน่วย,...,560.0,True,<NA>,34630,2026-01-26,372.100,OK,SYP,SYP-3TAR6902-346,TAR
1232656,2026-02-14,1.0,0,3TAR6902-346,11051952,เซ็นเซอร์ ABS หน้า ปาเจโร่ 2012 LH,1.0,Y,1.0,อัน,...,2450.0,True,<NA>,34631,2026-02-13,1501.500,OK,SYP,SYP-3TAR6902-346,TAR
1232657,2026-02-16,1.0,2,33K69-0001186,13013211,หลอดไฟในกันชน เขี้ยวตรง 12 V. 2 จุด สีส้ม,1.0,N,2.0,หลอด*,...,60.0,True,<NA>,34632,2026-02-06,16.000,OK,SYP,SYP-33K69-0001186,UNKNOWN
1232658,2026-02-16,1.0,2,33K69-0001187,08056145,"คอยล์เย็น VIOS,YARIS 2013",1.0,N,1.0,ลูก,...,1150.0,True,<NA>,34633,2025-11-01,747.665,OK,SYP,SYP-33K69-0001187,UNKNOWN


In [30]:
df_unknown = syp_sales_enriched[syp_sales_enriched["LAST_PURCHASE_COST"].isna()]
df_unknown


,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,CANCELED,PRICE_NUM,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH
50,20050,2,SJ,2025-06-25 00:00:00,1,2025-06-25,3K68-0000032,10,<NA>,70010011,...,N,400.0,-400.0,True,<NA>,50,NaT,NaN,UNKNOWN,SYP
198,70107,2,SJ,2025-07-01 00:00:00,1,2025-07-01,3K68-0000122,10,<NA>,13010000,...,N,1000.0,1000.0,True,<NA>,198,NaT,NaN,UNKNOWN,SYP
221,70133,2,SJ,2025-07-01 00:00:00,1,2025-07-01,3K68-0000135,50,<NA>,13010000,...,N,1000.0,-1000.0,True,<NA>,221,NaT,NaN,UNKNOWN,SYP
239,70155,2,SJ,2025-07-01 00:00:00,1,2025-07-01,3K68-0000143,20,<NA>,70010011,...,N,400.0,-400.0,True,<NA>,239,NaT,NaN,UNKNOWN,SYP
452,100340,2,SJ,2025-07-04 00:00:00,1,2025-07-04,3K68-0000193,10,<NA>,70010011,...,N,400.0,-400.0,True,<NA>,452,NaT,NaN,UNKNOWN,SYP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34100,1977472,2,SJ,2026-02-12 00:00:00,1,2026-02-12,33K69-0001113,10,<NA>,13010000,...,N,1000.0,1000.0,True,<NA>,34100,NaT,NaN,UNKNOWN,SYP
34102,1977474,2,SJ,2026-02-12 00:00:00,1,2026-02-12,33K69-0001114,10,<NA>,13010000,...,N,500.0,500.0,True,<NA>,34102,NaT,NaN,UNKNOWN,SYP
34266,1977653,2,SJ,2026-02-13 00:00:00,1,2026-02-13,32K69-0001132,30,<NA>,13010000,...,N,1000.0,-1000.0,True,<NA>,34266,NaT,NaN,UNKNOWN,SYP
34478,1987610,2,SJ,2026-02-14 00:00:00,1,2026-02-14,33K69-0001157,20,<NA>,13051932,...,N,5.0,10.0,True,<NA>,34478,NaT,NaN,UNKNOWN,SYP


In [31]:
pidet = data["raw_hq_pidet_purchase_lines.csv"].copy()
pidet_filtered = pidet[pidet["BCODE"] == "13010000"].copy()

pidet_filtered[["BCODE", 'BILLDATE', "QTY", "MTP", "PRICE", "AMOUNT"]]

,BCODE,BILLDATE,QTY,MTP,PRICE,AMOUNT


In [32]:
df_tfv_syp = sales_all[
    (sales_all["BILLTYPE_STD"] == "TAR") &
    (sales_all["BRANCH"] == "SYP")
]

df_tfv_syp

,BILLDATE,BILLTYPE,JOURMODE,BILLNO,BCODE,DETAIL,STATUS,ISVAT,QTY,UI,...,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH,BRANCH_BILLNO,BILLTYPE_STD
1198035,2025-06-23,1.0,0,3TAR6806-001,15018750,ลูกปืน 30-72-19 6306 2RS มีฝายาง,1.0,Y,1.0,ตับ,...,170.0,True,<NA>,10,2025-05-15,109.35000,OK,SYP,SYP-3TAR6806-001,TAR
1198036,2025-06-23,1.0,0,3TAR6806-001,15013500,ลูกปืน (ล้อหน้า L2600) 6205 2RSCM(ล้อหน้า L,1.0,Y,1.0,ตับ,...,85.0,True,<NA>,11,2025-06-13,51.40000,OK,SYP,SYP-3TAR6806-001,TAR
1198037,2025-06-23,1.0,0,3TAR6806-001,13022630,"สายพาน 52"" 12.5x1350",1.0,Y,1.0,เส้น,...,190.0,True,<NA>,12,2025-05-05,116.91250,OK,SYP,SYP-3TAR6806-001,TAR
1198038,2025-06-23,1.0,0,3TAR6806-001,14050200,สีสเปรย์ #36(No.300) บอร์นเงิน,1.0,Y,1.0,ก.ป.,...,45.0,True,<NA>,13,2025-06-10,30.37375,OK,SYP,SYP-3TAR6806-001,TAR
1198039,2025-06-23,1.0,0,3TAR6806-001,22050259,น้ำมันเบรค Brembo 0.5 L,1.0,Y,1.0,กป.,...,110.0,True,<NA>,14,2024-11-16,81.00000,OK,SYP,SYP-3TAR6806-001,TAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232652,2026-02-14,1.0,0,3TAR6902-346,22050207,น.ม.ค.Delo Gold 18LT 20W-50,1.0,Y,1.0,ถัง*,...,2030.0,True,<NA>,34627,2025-12-20,1822.43000,OK,SYP,SYP-3TAR6902-346,TAR
1232653,2026-02-14,1.0,0,3TAR6902-346,21050343,แบตเตอรี่มอเตอร์ไซ์ 12V 5An/10HR YTZ5S O,1.0,Y,1.0,หน่วย,...,380.0,True,<NA>,34628,2026-01-30,240.18700,OK,SYP,SYP-3TAR6902-346,TAR
1232654,2026-02-14,1.0,0,3TAR6902-346,13052513,เข็มขัดรัดท่อยาง-สแตนเลสกว้าง 40-55 (1.5/8-2.1...,1.0,Y,1.0,ตัว*,...,25.0,True,<NA>,34629,2025-10-09,13.88000,OK,SYP,SYP-3TAR6902-346,TAR
1232655,2026-02-14,1.0,0,3TAR6902-346,03052667,"ลูกหมากปีกนกล่าง 2WD-4WD D-MAX,MU7.04-13,MU-X",1.0,Y,1.0,หน่วย,...,560.0,True,<NA>,34630,2026-01-26,372.10000,OK,SYP,SYP-3TAR6902-346,TAR


In [33]:
sales_filtered = sales_all[
    (sales_all["JOURMODE"] == 0) &
    (~sales_all["BILLNO"].astype("string").str.contains("TAR", na=False)) &
    (sales_all["BILLDATE"].dt.year == 2025)
].copy()



sales_filtered

,BILLDATE,BILLTYPE,JOURMODE,BILLNO,BCODE,DETAIL,STATUS,ISVAT,QTY,UI,...,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH,BRANCH_BILLNO,BILLTYPE_STD
985753,2025-01-01,1.0,0,TAD6801-001,22023880,จารบีเทรนทอง HT 5Kg TRANE,1.0,Y,1.0,กป.,...,700.0,True,<NA>,985753,2024-11-14,595.794750,OK,HQ,HQ-TAD6801-001,TAD
985754,2025-01-01,1.0,0,TAD6801-001,21040180,แบตเตอรี่ FB N100 FB,1.0,Y,2.0,ลูก,...,7100.0,True,<NA>,985754,2024-12-19,2953.267000,OK,HQ,HQ-TAD6801-001,TAD
985755,2025-01-01,1.0,0,TR6801-006,14050201,สีสเปรย์ #14(No.226) ส้ม,1.0,Y,1.0,ก.ป.,...,50.0,True,<NA>,985755,2024-05-17,30.374167,OK,HQ,HQ-TR6801-006,TR
985756,2025-01-01,1.0,0,TR6801-006,13017610,"สายพาน 37.5"" 12.5x975",1.0,Y,1.0,เส้น,...,130.0,True,<NA>,985756,2024-12-24,84.958333,OK,HQ,HQ-TR6801-006,TR
985757,2025-01-01,1.0,0,TR6801-006,13014620,"สายพาน 48"" 12.5x1250 แอร์MTXมีP",1.0,Y,2.0,เส้น,...,360.0,True,<NA>,985757,2024-12-05,107.891250,OK,HQ,HQ-TR6801-006,TR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014526,2025-03-09,1.0,0,TAD6803-169,22010003,น้ำมันเบรคเชลล์ บ. 1LT DOT3 SHELL,1.0,Y,2.0,กป.,...,380.0,True,<NA>,1014526,2025-01-31,161.215000,OK,HQ,HQ-TAD6803-169,TAD
1014527,2025-03-09,1.0,0,TAD6803-169,19050021,"แกนสกรู ไม่หัว NC 5/8-2"" VEE",1.0,Y,10.0,ตัว,...,150.0,True,<NA>,1014527,2024-05-30,4.500000,OK,HQ,HQ-TAD6803-169,TAD
1014972,2025-03-10,1.0,0,TAD6803-206,22010037,สเปรย์ทำความสะอาดระบบเบรค CLEANUP 500ml. BENDIX,1.0,Y,1.0,กป.*,...,170.0,True,<NA>,1014972,2025-01-30,102.053333,OK,HQ,HQ-TAD6803-206,TAD
1014973,2025-03-10,1.0,0,TAD6803-206,22051820,น้ำยาหม้อน้ำ มิตซูบิชิ 5L แท้,1.0,Y,1.0,กป.*,...,390.0,True,<NA>,1014973,2025-02-15,276.500000,OK,HQ,HQ-TAD6803-206,TAD


In [34]:
print(sales_filtered["BILLNO"].to_list())


['TAD6801-001', 'TAD6801-001', 'TR6801-006', 'TR6801-006', 'TR6801-006', 'TR6801-006', 'TR6801-007', 'TR6801-007', 'TR6801-007', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-010', 'TR6801-010', 'TR6801-010', 'TR6801-010', 'TR6801-011', 'TR6801-012', 'TR6801-012', 'TR6801-013', 'TR6801-014', 'TR6801-014', 'TR6801-015', 'TR6801-015', 'TR6801-015', 'TAD6801-033', 'TR6801-020', 'TR6801-020', 'TR6801-021', 'TR6801-021', 'TR6801-021', 'TAD6801-042', 'TAD6801-042', 'TR6801-023', 'TAD6801-063', 'TAD6801-063', 'TAD6801-064', 'TAD6801-064', 'TAD6801-064', 'TAD6801-064', 'TAD6801-064', 'TAD6801-064', 'TAD6801-065', 'TAD6801-065', 'TAD6801-065', 'TAD6801-065', 'TAD6801-066', 'TAD6801-066', 'TAD6801-066', 'TR6801-024', 'TAD6801-067', 'TR6801-025', 'TAD6801-067', 'TR6801-033', 'TR6801-033', 'TR6801-033', 'TR6801-033', 'TR6801-033', 'TR6801-034', 'TR6801-034', 'TR6801-034', 'TR6801-034', 'TR6801-035', 'TAD6801-113', 'TAD6801-1